## **Demo Notebook**

This quick notebook will show how to

* load config
* create the dataset
* make a model
* load pretrained weights

This notebook should be run directly from the `scripts/` directory.

### **Config**

Make sure to change the following paths
* `DATASET_DIR`
* `LABELS_DIR`

In [1]:
from hydra import core, initialize, compose
from omegaconf import OmegaConf
import torch

device = torch.device('cuda:0')


# CHANGE ME
DATASET_DIR = '/shares/datasets/nuscenes/v1.0-trainval'
LABELS_DIR = '/shares/datasets/cvt_labels_nuscenes'


core.global_hydra.GlobalHydra.instance().clear()        # required for Hydra in notebooks

initialize(config_path='../config')

# Add additional command line overrides
cfg = compose(
    config_name='config',
    overrides=[
        'experiment.save_dir=../logs/',                 # required for Hydra in notebooks
        '+experiment=cvt_nuscenes_vehicle',
        f'data.dataset_dir={DATASET_DIR}',
        f'data.labels_dir={LABELS_DIR}',
        'data.version=v1.0-trainval',
        'loader.batch_size=1',
    ]
)

# resolve config references
OmegaConf.resolve(cfg)

print(list(cfg.keys()))

['experiment', 'loader', 'optimizer', 'scheduler', 'trainer', 'model', 'data', 'visualization', 'loss', 'metrics']


/tmp/ipykernel_1994907/897686792.py:15: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../config')


In [2]:
torch.cuda.is_available()

True

### **Dataset Setup**

In [14]:
import torch
import numpy as np

from cross_view_transformer.common import setup_experiment, load_backbone


# Additional splits can be added to cross_view_transformer/data/splits/nuscenes/
SPLIT = 'val_qualitative_000'
SUBSAMPLE = 5


model, data, viz = setup_experiment(cfg)

dataset = data.get_split(SPLIT, loader=False)
dataset = torch.utils.data.ConcatDataset(dataset)
dataset = torch.utils.data.Subset(dataset, range(0, len(dataset), SUBSAMPLE))

loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2)

print(len(dataset))

Loaded pretrained weights for efficientnet-b4
49


In [15]:
current = next(iter(loader))

In [17]:
current['bev'].shape  

torch.Size([4, 12, 200, 200])

### **Network Setup**

In [5]:
from pathlib import Path

# Download a pretrained model (13 Mb)
MODEL_URL = 'https://www.cs.utexas.edu/~bzhou/cvt/cvt_nuscenes_vehicles_50k.ckpt'
CHECKPOINT_PATH = '../logs/cvt_nuscenes_vehicles_50k.ckpt'

!wget $MODEL_URL -O $CHECKPOINT_PATH


if Path(CHECKPOINT_PATH).exists():
    network = load_backbone(CHECKPOINT_PATH)
else:
    network = model.backbone

    print(f'{CHECKPOINT_PATH} not found. Using randomly initialized weights.')

--2023-06-14 15:26:03--  https://www.cs.utexas.edu/~bzhou/cvt/cvt_nuscenes_vehicles_50k.ckpt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.cs.utexas.edu (www.cs.utexas.edu)... 128.83.120.48
Connecting to www.cs.utexas.edu (www.cs.utexas.edu)|128.83.120.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13407137 (13M)
Saving to: ‘../logs/cvt_nuscenes_vehicles_50k.ckpt’

../logs/cvt_nuscene 100%[===================>]  12.79M  84.7MB/s    in 0.2s    

2023-06-14 15:26:03 (84.7 MB/s) - ‘../logs/cvt_nuscenes_vehicles_50k.ckpt’ saved [13407137/13407137]

Loaded pretrained weights for efficientnet-b4


### **Run the Model**

This cell will save the model predictions and render them as a gif.

Note: colors will be ugly since gif only supports 256 colors total

In [6]:
# load IOU metric
from cross_view_transformer.metrics import IoUMetric

iou_config = OmegaConf.load('/home/dxa230028/Documents/cross_view_transformer/config/data/nuscenes_vehicle.yaml')

iou_metric = IoUMetric(label_indices=iou_config['data']['label_indices']).to(device)


In [8]:
import torch
import time
import imageio
import ipywidgets as widgets

GIF_PATH = './predictions.gif'

network.to(device)
network.eval()

images = list()

iou_metric.reset()


total_0_40_iou = 0
total_0_50_iou = 0

with torch.no_grad():
    for batch in loader:
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        pred = network(batch)['bev'].to(device)
        
        iou_metric.update(pred, batch)
        
        total_0_40_iou += iou_metric.compute()['@0.40']
        total_0_50_iou += iou_metric.compute()['@0.50']
        
        # visualization = np.vstack(viz(batch=batch, pred=pred))
        # images.append(visualization)

average_0_40_iou = total_0_40_iou / len(dataset)
average_0_50_iou = total_0_50_iou / len(dataset)

print(f"IOU Metric Value for threshold 0.40: {average_0_40_iou * 100}")
print(f"IOU Metric Value for threshold 0.50: {average_0_50_iou * 100}")

# # Save a gif
# duration = [1 for _ in images[:-1]] + [5 for _ in images[-1:]]
# imageio.mimsave(GIF_PATH, images, duration=duration)

# html = f'''
# <div align="center">
# <img src="{GIF_PATH}?modified={time.time()}" width="80%">
# </div>
# '''

# display(widgets.HTML(html))

IOU Metric Value for threshold 0.40: 37.701858671344056
IOU Metric Value for threshold 0.50: 33.51586375917707
